##### Import data

In [1]:
from util_copy import load_fabric_data, extract_label_grouping, extract_label_grouping, load_fabric_images
import numpy as np
from matplotlib import image

In [2]:

path = r"C:/Users/Administrator/Desktop/PRML/Project/fabric_data/label_json/**/**.json"

fids, fdata = load_fabric_data(path)
ftype1, ftype2 = extract_label_grouping(fdata)


In [3]:
path = r"C:/Users/Administrator/Desktop/PRML/Project/fabric_data/temp/"
labels, imgs = load_fabric_images(path, fids, fdata, ftype2)


C:/Users/Administrator/Desktop/PRML/Project/fabric_data/temp/**/**.jpg


In [4]:
print(len(labels))
print(imgs[2892].shape)

3371
(400, 400, 6)


In [5]:
n_samples = len(imgs)
print("Number of samples:", n_samples)

Number of samples: 3371


##### Padding
- https://blog.csdn.net/wuzqChom/article/details/74785643
- https://stackoverflow.com/questions/47697622/cnn-image-resizing-vs-padding-keeping-aspect-ratio-or-not/49882055#49882055
- https://stackoverflow.com/questions/43391205/add-padding-to-images-to-get-them-into-the-same-shape

In [6]:
import cv2

In [7]:
imgs = [cv2.resize(img, dsize=(200, 200)) for img in imgs]

##### Split data

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
train_images, test_images, train_labels, test_labels = train_test_split(imgs, labels, test_size=0.3, random_state=1)

In [ ]:
# print(len(train_images))

In [10]:
print("#Training data: {}\n#Testing data: {}\n#Class: {}".format(len(train_images), len(test_images), len(set(train_labels))))

#Training data: 2359
#Testing data: 1012
#Class: 9


##### Normalize data

In [11]:
train_images, test_images, train_labels, test_labels = np.array(train_images), np.array(test_images), np.array(train_labels), np.array(test_labels)
# train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
#train_images.shape

##### Generate fake data

##### Create model
- https://www.tensorflow.org/tutorials/images/classification

In [12]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [13]:
input_shape = (200, 200, 6)

In [24]:
model = models.Sequential()
model.add(layers.Conv2D(96, (11, 11), activation='relu', input_shape = input_shape))
model.add(layers.MaxPooling2D(pool_size= (5, 5), strides=(3,3)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size= (5, 5), strides=(3,3)))
model.add(layers.Conv2D(300, (3, 3), activation='relu'))
model.add(layers.Conv2D(300, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size= (4, 4), strides=(3,3)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(9))

In [26]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 190, 190, 96)      69792     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 62, 62, 96)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 60, 60, 256)       221440    
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 19, 19, 256)       0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 17, 17, 300)       691500    
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 15, 15, 300)       810300    
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 4, 4, 300)        

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

Epoch 1/10
37/74 [==============>...............] - ETA: 5:19 - loss: 19.5708 - accuracy: 0.1022

##### Testing

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)